In [3]:
from typing import List

from langchain_openai import ChatOpenAI
from pydantic import BaseModel

from browser_use import Agent, Controller

# Define the output format as a Pydantic model
class Post(BaseModel):
	post_title: str
	post_url: str
	num_comments: int
	hours_since_post: int


class Posts(BaseModel):
	posts: List[Post]


controller = Controller(output_model=Posts)


async def main():
	task = 'Go to hackernews show hn and give me the first  5 posts'
	model = ChatOpenAI(model='gpt-4o-mini')
	agent = Agent(task=task, llm=model, controller=controller)

	history = await agent.run()

	result = history.final_result()
	if result:
		parsed: Posts = Posts.model_validate_json(result)

		for post in parsed.posts:
			print('\n--------------------------------')
			print(f'Title:            {post.post_title}')
			print(f'URL:              {post.post_url}')
			print(f'Comments:         {post.num_comments}')
			print(f'Hours since post: {post.hours_since_post}')
	else:
		print('No result')


await main()

INFO     [agent] 🚀 Starting task: Go to hackernews show hn and give me the first  5 posts
INFO     [agent] 📍 Step 1
INFO     [agent] 🤷 Eval: Unknown - The page is empty and I need to navigate to Hacker News to proceed.
INFO     [agent] 🧠 Memory: 0 out of 5 posts retrieved from Hacker News. Need to navigate to Hacker News to start the task.
INFO     [agent] 🎯 Next goal: Search for Hacker News.
INFO     [agent] 🛠️  Action 1/1: {"search_google":{"query":"Hacker News Show HN"}}
INFO     [controller] 🔍  Searched for "Hacker News Show HN" in Google
INFO     [agent] 📍 Step 2
INFO     [agent] 🤷 Eval: Unknown - The search results are displayed without navigation.
INFO     [agent] 🧠 Memory: 0 out of 5 posts retrieved. I need to navigate to the actual Hacker News site to retrieve the posts.
INFO     [agent] 🎯 Next goal: Click on the Hacker News link.
INFO     [agent] 🛠️  Action 1/1: {"click_element":{"index":9}}
INFO     [controller] 🖱️  Clicked button with index 9: Terms
INFO     [agent] 📍 Step 